In [1]:
import mxnet as mx
import gluonnlp as nlp

ctx = mx.gpu()

In [2]:
# RNNs MXNet Implementation Example
class RNNModel(mx.gluon.Block):
    """
    A basic RNN Model
    """
    def __init__(self, num_hidden, num_layers, embed_size, **kwargs):
        
        super(RNNModel, self).__init__(**kwargs)
        
        self.rnn = mx.gluon.rnn.RNN(
            num_hidden,
            num_layers,
            input_size=embed_size)
        
        
    def forward(self, inputs, hidden):
        output, hidden = self.rnn(inputs, hidden)
        return output, hidden

In [3]:
# RNN with 3 hidden cells, 1 layer and expecting inputs with 20 embeddings
rnn = RNNModel(3, 1, 20)
rnn.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

In [4]:
# Initial Hidden Values
hidden_initial = mx.nd.random.randn(1, 1, 3, ctx=ctx)

In [5]:
# Inputs generation
# Let's input 3 words 
# Each word is passed through a 20-component embedding model
inputs = []

# Each word representation
inputs.append(mx.nd.random.randn(3, 1, 20, ctx=ctx))
inputs.append(mx.nd.random.randn(3, 1, 20, ctx=ctx))
inputs.append(mx.nd.random.randn(3, 1, 20, ctx=ctx))

In [6]:
rnn_hidden = hidden_initial
outputs = []

for index in range(3):
    rnn_output, rnn_hidden = rnn(inputs[index], rnn_hidden)
    outputs.append(rnn_output)

print(rnn_output, rnn_hidden)

Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/mxnet_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2022-07-28 11:51:11.543 ip-172-31-28-47:19469 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-07-28 11:51:11.572 ip-172-31-28-47:19469 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.

[[[0.36091858 0.         0.        ]]

 [[0.7694626  1.3475145  0.5260414 ]]

 [[0.         0.         0.        ]]]
<NDArray 3x1x3 @gpu(0)> [
[[[0. 0. 0.]]]
<NDArray 1x1x3 @gpu(0)>]


In [7]:
# LSTMs MXNet Implementation Example
class LSTMModel(mx.gluon.Block):
    """
    A basic LSTM Model
    """
    def __init__(self, num_hidden, num_layers, embed_size, **kwargs):
        
        super(LSTMModel, self).__init__(**kwargs)
        
        self.lstm = mx.gluon.rnn.LSTM(
            num_hidden,
            num_layers,
            input_size=embed_size)
        
        
    def forward(self, inputs, hidden):
        output, hidden = self.lstm(inputs, hidden)
        return output, hidden

In [8]:
# LSTM with 3 hidden cells, 1 layer and expecting inputs with 20 embeddings
lstm = LSTMModel(3, 1, 20)
lstm.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

In [9]:
# Initial Hidden Values
hidden_initial = mx.nd.random.randn(1, 1, 3, ctx=ctx)

# Initial State Values
state_initial = mx.nd.random.randn(1, 1, 3, ctx=ctx)

In [11]:
lstm_hidden = [hidden_initial, state_initial]
outputs = []

for index in range(3):
    lstm_output, lstm_hidden = lstm(inputs[index], lstm_hidden)
    outputs.append(lstm_output)

print(lstm_output, lstm_hidden)


[[[ 0.11808932  0.00715288 -0.17337681]]

 [[ 0.05276631 -0.31671402  0.26189104]]

 [[-0.2773834   0.00210142  0.2504536 ]]]
<NDArray 3x1x3 @gpu(0)> [
[[[-0.2773834   0.00210142  0.2504536 ]]]
<NDArray 1x1x3 @gpu(0)>, 
[[[-0.42638525  0.0045678   0.40797684]]]
<NDArray 1x1x3 @gpu(0)>]


In [53]:
# Transformers MXNet Implementation Example
# Transformer with 6 layers (encoder and decoder), 2 parallel heads, and expecting inputs with 20 embeddings
transformer_encoder, transformer_decoder, _ = nlp.model.transformer.get_transformer_encoder_decoder(
    num_layers=6,
    num_heads=2,
    units=20)

transformer_encoder.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
transformer_decoder.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

In [54]:
# Only encoder shown
encoded_inputs, _ = transformer_encoder(inputs[0])

print(encoded_inputs)


[[[ 0.83209103 -1.6060255   0.22104202  0.61695844  1.3568467
   -0.32727766  0.91617656  0.10976635 -0.9734795  -2.0243998
   -0.3601701   0.6375824  -0.7103551  -0.5400416   1.0467256
   -0.48623165 -0.74315417 -0.5523693   2.1838753   0.40243995]]

 [[ 0.8831734  -0.9573856   0.91161126  0.297851    0.1370347
   -0.6730614   0.6583741   0.519483   -1.2481059  -1.0314509
    0.52972645  0.21611245 -1.0184485  -0.18149762 -0.18714839
   -0.46389332 -2.1304226   0.01332407  2.3436754   1.3810486 ]]

 [[ 0.10657646  0.41115397 -1.4852034  -0.40792608 -0.36205494
   -0.65480286 -1.2349089  -0.35988408 -0.16835214  3.1068172
   -0.88195777 -0.2612006   0.30937973  0.4125706   0.600156
   -0.81594104  0.7318146   0.38038185 -0.8467938   1.4201748 ]]]
<NDArray 3x1x20 @gpu(0)>
